## Genralization

 setup

In [5]:
from collections import namedtuple, Counter, defaultdict
import random
import math
import functools 
infinity = math.inf

game class

In [7]:
class Game:
    def actions(self,state):
        raise NotImplementedError
    def result(self,state,move):
        raise NotImplementedError
    def is_terminal(self, state):
        raise NotImplementedError
    def utility(self, state , player):
        raise NotImplementedError

Minimax Search algorithm

In [9]:
def minimax_search(game, state):
    
    player = state.to_move
    
    def max_value(state):
        if game.is_terminal(state) :
            return game.utility(state , player), None
        v, move = -infinity, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a))
            if v2 > v:
                v , move = v2, a
        return v, move

    def min_value(state):
        if game.is_terminal(state):
            return game.utility(state,player), None
        v, move = +infinity , None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a))
            if v2 < v:
                v , move = v2 , a
        return v, move

    return max_value(state)


alphabeta Search

In [11]:
def alphabeta_search(game, state):
    
    player = state.to_move

    def max_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -infinity, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a) , alpha, beta)
            if v2 > v:
                v, move = v2 , a
                alpha = max(alpha, v)
            if v >= beta:
                return v, move
        return v, move

    def min_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state , player), None
        v, move = +infinity, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a), alpha, beta)
            if v2 < v:
                v, move = v2, a
                beta = min(v, beta)
            if v <= alpha:
                return v, move
        return v, move

    return max_value(state, -infinity, +infinity)

minimax search : transposition table version

In [13]:
cache = functools.lru_cache(10**6)

def minimax_search_tt(game, state):
    
    player = state.to_move

    @cache
    def max_value(state):
        if game.is_terminal(state) :
            return game.utility(state , player), None
        v , move = -infinity, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a))
            if v2 > v:
                v , move = v2 , a 
        return v, move
        
    @cache
    def min_value(state):
        if game.is_terminal(state):
            return game.utility(state,player), None
        v, move = +infinity , None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a))
            if v2 < v:
                v , move = v2 , a
        return v, move

    return max_value(state)

alphabeta search : transpostion version

In [36]:
def cache1(function):
    #to consider only the first argument (ignoring alpha, beta)
    cache = {}
    def wrapper(x, *args):
        if x not in cache:
            cache[x] = function(x, *args)
        return cache[x]
    return wrapper
    
def alphabeta_search_tt(game, state):
    
    player = state.to_move

    @cache1
    def max_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -infinity, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a) , alpha, beta)
            if v2 > v:
                v, move = v2 , a
                alpha = max(alpha, v)
            if v >= beta:
                return v, move
        return v, move

    @cache1
    def min_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state , player), None
        v, move = +infinity, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a), alpha, beta)
            if v2 < v:
                v, move = v2, a
                beta = min(v, beta)
            if v <= alpha:
                return v, move
        return v, move

    return max_value(state, -infinity, +infinity)

alphabeta search : using heuristic cutoff and eval

In [39]:
def cutoff_depth(d):
    """A cutoff function that searches to depth d."""
    return lambda game, state, depth: depth > d

def alphabeta_search_h(game, state, cutoff = cutoff_depth(6), h = lambda s, p: 0):
    """Search game to determine best action; use alpha-beta pruning.
    As in [Figure 5.7], this version searches all the way to the leaves."""

    player = state.to_move

    @cache1
    def max_value(state, alpha, beta, depth):
        if game.is_terminal(state):
            return game.utility(state, player), None
        if cutoff(game, state, depth):
            return h(state, player), None
        v, move = -infinity, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a), alpha, beta, depth+1)
            if v2 > v:
                v, move = v2, a
                alpha = max(alpha, v)
            if v >= beta:
                return v, move
        return v, move

    @cache1
    def min_value(state, alpha, beta, depth):
        if game.is_terminal(state):
            return game.utility(state, player), None
        if cutoff(game, state, depth):
            return h(state, player), None
        v, move = +infinity, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a), alpha, beta, depth + 1)
            if v2 < v:
                v, move = v2, a
                beta = min(beta, v)
            if v <= alpha:
                return v, move
        return v, move

    return max_value(state, -infinity, +infinity, 0)

video game

In [42]:
def play_game(game, strategies : dict , verbose = True):
    state = game.initial
    while not game.is_terminal(state) :
        player = state.to_move
        move = strategies[player](game,state)
        state = game.result(state , move)
        if verbose:
            print('player' , player , 'move' , move)
            print(state)
    return state

## 1: TIC-TAC-TOE game

tictactoe class

In [46]:
class TicTacToe(Game):
    def __init__(self, height = 3 , width = 3 , k = 3):
        self.k = k
        self.initial = Board(height = height, width = width, to_move='X', utility=0)
        self.squares = {(x,y) for x in range(width) for y in range(height)}

    def actions(self,board):
        return self.squares - set(board)

    def result(self, board , square):
        player = board.to_move
        board = board.new({square: player}, to_move=('O' if player == 'X' else 'X'))
        win = k_in_row(board, player, square, self.k)
        board.utility = ( 0 if not win else +1 if player =='X' else -1)
        return board

    def utility(self, board , player):
        return board.utility if player == 'X' else -board.utility

    def is_terminal(self, board):
        return board.utility != 0 or len(self.squares) == len(board)

def k_in_row(board, player, square, k):
    """True if player has k pieces in a line through square."""
    def in_row(x, y, dx, dy): return 0 if board[x, y] != player else 1 + in_row(x + dx, y + dy, dx, dy)
    return any(in_row(*square, dx, dy) + in_row(*square, -dx, -dy) - 1 >= k
               for (dx, dy) in ((0, 1), (1, 0), (1, 1), (1, -1)))

state (board) class

In [49]:
class Board(defaultdict):
    """ - It is a sub-class of the dictionary class that returns a dictionary-like object 
        - The difference from dictionary is, It provides a default value for the key that does not exist and never raises a KeyError 
        - has a dict of {(x,y) : player} where player is 'X' or 'O' """
    empty = '.'
    off = '#'
    def __init__(self,width=8, height=8, to_move=None, **kwds):
        self.__dict__.update(width = width , height= height , to_move = to_move , **kwds)
        """ 
            b = Board(width=5, height=5, to_move='X', extra_info='test')
            {
                'width': 5,
                'height': 5,
                'to_move': 'X',
                'extra_info': 'test'
            }
        """
    def new(self, changes : dict , **kwds):
        board = Board(width=self.width, height=self.height, **kwds)
        board.update(self)
        board.update(changes)
        return board

    def __hash__(self): 
        """will be needed in caching """
        return hash(tuple(sorted(self.items()))) + hash(self.to_move)
        
    def __missing__(self, loc):
        """ will put . or # for the missing values in our dict """
        x, y = loc
        return self.empty if 0 <= x < self.width and 0 <= y < self.height else self.off
     
    def __repr__(self):
        """ customize the print of the our dict """
        def row(y): 
            return ' '.join(self[x, y] for x in range(self.width))
        return '\n'.join(map(row, range(self.height))) + '\n'

## 2: class ConnectFour

Connect 4 class

In [53]:
class ConnectFour(TicTacToe):
    
    def __init__(self): 
        super().__init__(width=7, height=6, k=4)

    def actions(self, board):
        """In each column you can play only the lowest empty square in the column."""
        return {(x, y) for (x, y) in self.squares - set(board)
                if y == board.height - 1 or (x, y + 1) in board}

players and test

In [56]:
def random_player(game , state):
    """ This player just chooses a random move.     player O """
    return random.choice(list(game.actions(state)))

def player(search_algo):
    """A game player who uses the specified search algorithm.    player X """
    return lambda game , state : search_algo(game, state)[1] # there was [1]

#%time play_game(TicTacToe(), dict(O = player(minimax_search), X = player(alphabeta_search)), verbose = True).utility
#play_game(ConnectFour(), dict(X=random_player, O=random_player), verbose=True).utility
#%time play_game(TicTacToe(), dict(O = player(minimax_search_tt), X = player(alphabeta_search_tt)), verbose = False).utility
#%time play_game(ConnectFour(), {'X':player(alphabeta_search_h), 'O':player(alphabeta_search_h)}).utility

## 3: Pac-Man non-vanilla

In [59]:
class PacMan(Game):
    def __init__(self, maze, x_pos, o_pos):

        food_positions = set()
        for y in range(len(maze)):
            for x in range(len(maze[0])):
                if maze[y][x] == 'c':
                    food_positions.add((x, y))
        
        self.initial = Maze(height=len(maze), width=len(maze[0]), to_move='X', 
                           utility=0, food=len(food_positions), food_positions=food_positions,
                           x_pos=x_pos, o_pos=o_pos, x_food=0, o_food=0)
        maze_data = {(x, y): maze[y][x] for y in range(len(maze)) for x in range(len(maze[0]))}
        self.food = len(food_positions)
        self.initial.update(maze_data)
        
    def actions(self, state):
        actions = set()
        
        player = state.to_move
        pos = state.x_pos if player == "X" else state.o_pos
        pos_e = state.o_pos if player == "X" else state.x_pos

        directions = [(-1, 0), (1, 0),  (0, -1), (0, 1)]  # UP, DOWN, LEFT, RIGHT

        for dx, dy in directions:
            new_pos = (pos[0] + dx, pos[1] + dy)
            
            if (
                0 <= new_pos[0] < state.width and
                0 <= new_pos[1] < state.height and
                state[new_pos] != '#' and
                new_pos != pos_e
            ):
                actions.add(new_pos)
        if len(actions) == 0:
            actions.add((pos[0],pos[1]))
        return actions
        
    def result(self, state, move):
        player = state.to_move
        old_pos = state.x_pos if player == 'X' else state.o_pos
        state = state.new({move: player}, to_move ='O' if player == 'X' else 'X', old_pos = old_pos)
        win = state.food == 0
        state.utility = (0 if (not win or (win and state.x_food==state.o_food)) else +1 if (player =='X' and state.x_food  > state.o_food) else -1)
        return state
        
    def is_terminal(self, state):
        return state.food == 0  

    def utility(self, state , player):
        return state.utility if player == 'X' else -state.utility



class Maze(defaultdict):
    empty = '.'
    off = '#'
    food = 'c'

    def __init__(self, width=8, height=8, to_move='X', **kwds):
        self.__dict__.update(width=width, height=height, to_move=to_move, **kwds)
        
    def new(self, changes: dict, old_pos=None, **kwds):
        maze = Maze(width=self.width, height=self.height, 
                    food=self.food, food_positions=set(self.food_positions),
                    x_food=self.x_food, o_food=self.o_food,
                    x_pos=self.x_pos, o_pos=self.o_pos, utility = self.utility,**kwds)
        
        # Copy all existing positions
        maze.update(self)
        
        # Get the new position from changes
        new_pos = list(changes.keys())[0]
        player = changes[new_pos]
        
        # Update player positions
        if player == 'X':
            maze.x_pos = new_pos
        else:
            maze.o_pos = new_pos
        
        # Handle food collection
        if new_pos in self.food_positions:
            maze.food -= 1
            maze.food_positions.remove(new_pos)
            if player == 'X':
                maze.x_food += 1
            else:
                maze.o_food += 1
        
        # Clear old position if it wasn't food
        if old_pos and old_pos not in self.food_positions:
            maze[old_pos] = self.empty
        
        # Apply the changes (player movement)
        maze.update(changes)
        
        return maze

    def __hash__(self):
        """Hash for caching, based on maze contents and current player"""
        return hash(tuple(sorted(self.items()))) + hash(self.to_move) + hash(tuple(sorted(self.food_positions)))

    def __repr__(self):
        """Customize the print of our dict"""
        def row(y): 
            return ' '.join(self[x, y] for x in range(self.width))
        return '\n'.join(map(row, range(self.height))) + '\n'

test it

In [62]:
maze = [
    ['#', '.', 'c'],
    ['O', 'c', 'X'],
]

def playeer(search_algo):
    return lambda game , state : search_algo(game, state)


game = PacMan(maze=maze, x_pos=(2,1), o_pos=(0,1))

#%time play_game(game, dict(X=player(minimax_search), O=playeer(random_player)), verbose=True).utility        

## 4:Pirates

In [73]:
class Pirates(Game):
    def __init__(self, grid, tresors, x, o):
        self.initial = Zone(width=len(grid[0]), height = len(grid),to_move = 'X',
                            x_score = 0, o_score = 0, utility = 0,
                            x_pos = x, o_pos = o, grid = grid, x_old= x, o_old = o)

        maze_data = {(x, y): grid[y][x] for y in range(len(grid)) for x in range(len(grid[0]))}
        self.initial.update(maze_data)
        self.tresors = tresors


    def actions(self, state):
        actions = set()
        
        player = state.to_move
        pos = state.x_pos if player == "X" else state.o_pos
        old_pos = state.x_old if player == "X" else state.o_old

        for dx, dy in [(-1, 0), (1, 0),  (0, -1), (0, 1)]:
            new_pos = (pos[0] + dx, pos[1] + dy)
            
            if (
                0 <= new_pos[0] < state.width and
                0 <= new_pos[1] < state.height and
                state[new_pos] != '#' and
                new_pos != old_pos
            ):
              
                actions.add(new_pos)

        return actions

    def result(self, state, move):
        player = state.to_move
        state = state.new({move: player}, 
                            to_move='O' if player == 'X' else 'X',
                            x_pos=state.x_pos,
                            o_pos=state.o_pos,
                            x_score=state.x_score,
                            o_score=state.o_score,
                            x_old = state.x_pos if player == 'X' else state.x_old,
                            o_old = state.o_pos if player == 'X' else state.x_old)
        
        # Check if game is over
        win = state.x_score + state.o_score == self.tresors
        state.utility = (0 if (not win or (win and state.x_score == state.o_score)) else 
                           +1 if (player == 'X' and state.x_score > state.o_score) 
                           else -1)
        return state
        
    def is_terminal(self, state):
        return state.utility != 0 or state.x_score + state.o_score == self.tresors
        
    def utility(self, state, player):
        return state.utility if player == 'X' else -state.utility




#-------------------------------STATE-----------------------------
class Zone(defaultdict):
    empty = '.'
    
    def __init__(self, width=8, height=8, to_move='X', x_score=0, o_score=0, utility=0, 
                 x_pos=None, o_pos=None, grid=None,x_old = None, o_old = None, **kwds):
        self.__dict__.update(width=width, height=height, to_move=to_move,
                           x_score=x_score, o_score=o_score, utility=utility,
                           x_pos=x_pos, o_pos=o_pos, grid=grid,x_old = x_old, o_old= o_old, **kwds)
        
    def new(self, changes: dict, **kwds):
        state = Zone(width=self.width, height=self.height, **kwds)
        state.update(self)
        state.update(changes)
        # Maintain the grid reference
        state.grid = self.grid.copy()
            

        # Update positions and scores
        new_pos = list(changes.keys())[0]
        player = changes[new_pos]
        
        cell_value = self[new_pos]
        for (x, y), val in changes.items():
            state.grid[y][x] = val  # Note: grid[y][x] not grid[x,y]
        if player == 'X':
            state.x_pos = new_pos
            state[self.x_pos] = self.empty
            if isinstance(cell_value, int):
                state.x_score += cell_value
        else:
            state.o_pos = new_pos
            state[self.o_pos] = self.empty
            if isinstance(cell_value, int):
                state.o_score += cell_value

        return state
        
    def __hash__(self):
        return hash(tuple(sorted(self.items()))) + hash(self.to_move) + hash(self.x_pos) + hash(self.o_pos)
    
    def __repr__(self):
        def row(y): 
            return ' '.join(str(self[x, y]) for x in range(self.width))
        return '\n'.join(map(row, range(self.height))) + '\n'
        


#--------------------------------HEURISITIC--------------------------
def h(state, player):
    diff_score = state.x_score - state.o_score
    if player == 'O':
        diff_score = -diff_score
    def manhattan(p, r):
      return abs(p[0] - r[0]) + abs(p[1] - r[1])

    def nearest_treasure_distance(pos):
        dists = []
        # Iterate through grid rows and columns
        for y in range(state.height):
            for x in range(state.width):
                val = state.grid[y][x]  # Access grid as 2D list
                if isinstance(val, int):  # Check if it's a treasure
                    dists.append(manhattan(pos, (x, y)))
        return min(dists) if dists else 0    
    diff_dist = nearest_treasure_distance(state.x_pos)  - nearest_treasure_distance(state.o_pos)
    if player == 'O':
        diff_dist = -diff_dist
    return 5 * diff_score + diff_dist

test

In [ ]:
zone = [
    ['.', 1, 'O', '#'],
    ['#', '.', 2, '.'],
    [3, '.', 'X', 1]
]

game = Pirates(grid = zone, tresors = 7 , x=(2,2), o=(2,0))

%time play_game(game, dict(X=player(lambda g, s: alphabeta_search_h(g, s, h=h)),O=playeer(random_player))).utility   